
<h2>2. Movie Recommendation System with SVD:</h2>
<ul>
<li>Build a movie recommendation system using collaborative filtering with SVD.</li>
<li>Apply SVD to factorize the user-item matrix and generate personalized movie recommendations based on a user's preferences.</li>
</ul>

<h3>Data processing</h3>

In [143]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [144]:
movies = pd.read_csv("data\\movies_metadata.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_3232\1979537588.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("data\\movies_metadata.csv")


In [145]:
movies.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
11430,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,76211,tt0040795,en,Sitting Pretty,Tacey and Harry King are a suburban couple wit...,...,1948-01-01,0.0,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Never was a baby-sitter like this!,Sitting Pretty,False,6.1,13.0
15285,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,22241,tt1263679,en,Firaaq,Firaaq is an Urdu word that means both separat...,...,2009-01-01,0.0,0.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Firaaq,False,5.0,1.0
21645,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",NaN,219288,tt1772262,en,Dragon Day,"When Duke Evans, out of work NSA analyst, is e...",...,2013-01-10,0.0,96.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The hack is the attack.,Dragon Day,False,3.2,5.0
21339,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,125164,tt1999243,pt,Rafa,Rafa is a 13 years old kid concerned with his ...,...,2012-05-10,0.0,25.0,"[{'iso_639_1': 'pt', 'name': 'Português'}]",Released,NaN,Rafa,False,6.6,5.0
17291,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",NaN,94241,tt0034242,en,Sundown,Englishmen fighting Nazis in Africa discover a...,...,1941-10-31,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,She was too dangerous to love!,Sundown,False,4.8,2.0


In [146]:
movies.shape

(45466, 24)

In [147]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

<h4>Handle missing data</h4>

In [148]:
movies.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [149]:
movies.loc[movies['title'].isna(), 'title'] = movies.loc[movies['title'].isna()]['original_title']

In [150]:
movies.loc[movies['vote_average'].isna(), 'vote_average'] = 0

In [151]:
movies.loc[movies['vote_count'].isna(), 'vote_count'] = 0

In [152]:
movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [153]:
movies.sample(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
2416,False,"{'id': 1709, 'name': 'Planet of the Apes Origi...",5800000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://www.foxmovies.com/movies/planet-of-the-...,871,tt0063442,en,Planet of the Apes,"An U.S. Spaceship lands on a desolate planet,...",...,1968-02-07,33395426.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Somewhere in the Universe, there must be somet...",Planet of the Apes,False,7.5,958.0
26271,False,NaN,4500000,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,35153,tt0473364,en,The Heavy,Sibling rivalry turned into betrayal between t...,...,2010-04-08,0.0,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Heavy,False,4.9,20.0
37771,False,NaN,0,"[{'id': 37, 'name': 'Western'}]",NaN,207178,tt0044838,en,The Lion and the Horse,"After selling it to a cruel rodeo owner, a cow...",...,1952-05-16,0.0,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,King of the wild beasts against king of wild h...,The Lion and the Horse,False,5.0,1.0


In [154]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [155]:
movies = movies[['id', 'title','genres', 'status', 'overview', 'popularity', 'runtime', 'original_language', 'production_companies', 'production_countries', 'vote_average', 'vote_count']]

In [156]:
movies.duplicated().sum()

13

In [157]:
movies.drop_duplicates(keep='first', inplace=True)

In [158]:
movies.duplicated().sum()

0

In [159]:
movies.isna().sum()

id                        0
title                     0
genres                    0
status                   87
overview                954
popularity                5
runtime                 263
original_language        11
production_companies      3
production_countries      3
vote_average              0
vote_count                0
dtype: int64

In [160]:
movies.sample(3)

,id,title,genres,status,overview,popularity,runtime,original_language,production_companies,production_countries,vote_average,vote_count
19739,97430,The Man with the Iron Fists,"[{'id': 28, 'name': 'Action'}]",Released,"In feudal China, a blacksmith who makes weapon...",8.310945,96.0,en,"[{'name': 'Arcade Pictures', 'id': 11112}, {'n...","[{'iso_3166_1': 'HK', 'name': 'Hong Kong'}, {'...",5.2,369.0
4685,11313,Hearts in Atlantis,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",Released,A widowed mother and her son change when a mys...,6.425321,101.0,en,"[{'name': 'Village Roadshow Pictures', 'id': 7...","[{'iso_3166_1': 'AU', 'name': 'Australia'}, {'...",6.5,132.0
33314,31927,One of Us,"[{'id': 18, 'name': 'Drama'}]",Released,A few Palestinian guerilla's break into an Isr...,0.000308,110.0,he,[],"[{'iso_3166_1': 'IL', 'name': 'Israel'}]",5.0,1.0


In [161]:
movies.original_language.value_counts()

original_language
en       32262
fr        2437
it        1529
ja        1350
de        1079
         ...  
zu           1
qu           1
104.0        1
la           1
si           1
Name: count, Length: 92, dtype: int64

In [162]:
movies.loc[movies['original_language'].isna(), 'original_language'] = 'en'

In [163]:
movies.isna().sum()

id                        0
title                     0
genres                    0
status                   87
overview                954
popularity                5
runtime                 263
original_language         0
production_companies      3
production_countries      3
vote_average              0
vote_count                0
dtype: int64

In [164]:
movies[movies['popularity'].isna()]

,id,title,genres,status,overview,popularity,runtime,original_language,production_companies,production_countries,vote_average,vote_count
19729,82663,Midnight Man,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,British soldiers force a recently captured IRA...,NaN,NaN,en,NaN,NaN,0.0,0.0
19730,1997-08-20,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'name': 'Carousel Productions', 'id': 11176}...",NaN,Released,NaN,NaN,104.0,False,6.0,0.0,0.0
29502,122662,マルドゥック・スクランブル 排気,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",NaN,Third film of the Mardock Scramble series.,NaN,NaN,ja,NaN,NaN,0.0,0.0
29503,2012-09-29,"[{'iso_639_1': 'ja', 'name': '日本語'}]","[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",NaN,Released,NaN,NaN,68.0,False,7.0,0.0,0.0
35586,249260,Avalanche Sharks,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...",NaN,A group of skiers are terrorized during spring...,NaN,NaN,en,NaN,NaN,0.0,0.0


In [165]:
movies.drop('runtime', inplace=True, axis=1)

In [166]:
movies.loc[movies['overview'].isna(), 'overview'] = 'Not thing'

In [167]:
movies.loc[movies['status'].isna(), 'status'] = 'Unknown'

In [168]:
movies.sample(5)

,id,title,genres,status,overview,popularity,original_language,production_companies,production_countries,vote_average,vote_count
3873,11983,Proof of Life,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Released,Alice hires a professional negotiator to obtai...,9.508691,en,"[{'name': 'Castle Rock Entertainment', 'id': 9...","[{'iso_3166_1': 'US', 'name': 'United States o...",6.1,211.0
38606,253484,1 Chance 2 Dance,[],Released,When a 17-year old aspiring dancer is uprooted...,1.212888,en,[],[],5.6,16.0
21667,245950,The Dragonphoenix Chronicles: Indomitable,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Released,A savage warrior escapes slavery and hunted by...,0.381124,el,"[{'name': 'Avalon Productions', 'id': 18970}]","[{'iso_3166_1': 'GR', 'name': 'Greece'}]",5.0,2.0
33843,335640,The Deal,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Released,Serial killer Kang-Cheon made several victims....,0.996272,ko,"[{'name': '9ers Entertainment', 'id': 41527}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",5.7,10.0
5499,9260,Welcome to Collinwood,"[{'id': 35, 'name': 'Comedy'}]",Released,Five hapless inner-city low-lifes attempt to b...,3.031934,en,"[{'name': 'Section Eight', 'id': 129}, {'name'...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",6.2,50.0


In [169]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45453 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45453 non-null  object 
 1   title                 45453 non-null  object 
 2   genres                45453 non-null  object 
 3   status                45453 non-null  object 
 4   overview              45453 non-null  object 
 5   popularity            45448 non-null  object 
 6   original_language     45453 non-null  object 
 7   production_companies  45450 non-null  object 
 8   production_countries  45450 non-null  object 
 9   vote_average          45453 non-null  float64
 10  vote_count            45453 non-null  float64
dtypes: float64(2), object(9)
memory usage: 4.2+ MB


In [170]:
movies.loc[movies['id'] == '2014-01-01', 'id'] = 201411

In [174]:
movies['id'] = movies['id'].astype(int)

In [173]:
movies.loc[movies['id'] == '1997-08-20','id'] = 199782

In [172]:
movies.loc[movies['id'] == '2012-09-29','id'] = 2012929

In [ ]:
movies.loc[movies['id'] == 201411, 'popularity'] = 0.0

In [ ]:
movies['popularity'] = movies['popularity'].astype(float)

In [185]:
movies.isna().sum()

id                      0
title                   0
genres                  0
status                  0
overview                0
popularity              4
original_language       0
production_companies    3
production_countries    3
vote_average            0
vote_count              0
dtype: int64

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.head(2)

,id,title,genres,status,overview,popularity,original_language,production_companies,production_countries,vote_average,vote_count
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Released,"Led by Woody, Andy's toys live happily in his ...",21.946943,en,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",7.7,5415.0
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Released,When siblings Judy and Peter discover an encha...,17.015539,en,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",6.9,2413.0


In [176]:
movies.drop(movies.loc[movies['genres'] == '[]'].index, inplace=True)

In [178]:
import ast

In [179]:
def get_genre(x):
  x = ast.literal_eval(x)
  res = []
  for item in x:
    res.append(item['name'])
  return ','.join(res)

In [186]:
movies['genres'] = movies['genres'].apply(get_genre)

In [187]:
movies.loc[movies['id'] == 199782]

,id,title,genres,status,overview,popularity,original_language,production_companies,production_countries,vote_average,vote_count
22364,199782,Springsteen & I,"Documentary,Music",Released,For 40 years Bruce Springsteen has influenced ...,1.5699,en,"[{'name': 'Scott Free Productions', 'id': 1645...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",7.5,11.0


In [183]:
movies.drop(index=19730, inplace=True)

In [188]:
movies.head(2)

,id,title,genres,status,overview,popularity,original_language,production_companies,production_countries,vote_average,vote_count
0,862,Toy Story,"Animation,Comedy,Family",Released,"Led by Woody, Andy's toys live happily in his ...",21.946943,en,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",7.7,5415.0
1,8844,Jumanji,"Adventure,Fantasy,Family",Released,When siblings Judy and Peter discover an encha...,17.015539,en,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",6.9,2413.0


In [195]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43010 entries, 0 to 45463
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    43010 non-null  int32  
 1   title                 43010 non-null  object 
 2   genres                43010 non-null  object 
 3   status                43010 non-null  object 
 4   overview              43010 non-null  object 
 5   popularity            43010 non-null  float64
 6   original_language     43010 non-null  object 
 7   production_companies  43007 non-null  object 
 8   production_countries  43007 non-null  object 
 9   vote_average          43010 non-null  float64
 10  vote_count            43010 non-null  int32  
dtypes: float64(2), int32(2), object(7)
memory usage: 3.6+ MB


In [190]:
movies['vote_count'] = movies['vote_count'].astype(int)

In [194]:
movies.loc[movies['popularity'].isna(), 'popularity'] = movies['popularity'].mean()

In [193]:
movies['popularity'] = movies['popularity'].astype(float)

In [192]:
movies.loc[movies['popularity'] == 'Beware Of Frost Bites', 'popularity'] = None

In [221]:
movies.sample(5)

,id,title,genres,status,overview,popularity,original_language,production_companies,production_countries,vote_average,vote_count
27152,297556,Justice League: Throne of Atlantis,"Science Fiction,Action,Animation,Adventure",Released,"After the events of Justice League: War, Ocean...",6.012379,en,"[{'name': 'DC Comics', 'id': 429}, {'name': 'W...","[{'iso_3166_1': 'US', 'name': 'United States o...",6.5,222
14461,20764,Sleep Dealer,"Drama,Science Fiction,Thriller",Released,"Set in a near-future, militarized world marked...",3.750365,en,"[{'name': 'Likely Story', 'id': 1785}]","[{'iso_3166_1': 'MX', 'name': 'Mexico'}, {'iso...",6.0,41
4389,49365,Lady in White,"Horror,Thriller",Released,Locked in a school closet during Halloween 196...,1.197890,en,"[{'name': 'New Sky Communications Inc.', 'id':...","[{'iso_3166_1': 'US', 'name': 'United States o...",6.3,25
31332,53914,The Story of Piera,Drama,Released,This sometimes confusing erotic drama about th...,1.448790,it,"[{'name': 'Sara Films', 'id': 876}, {'name': '...","[{'iso_3166_1': 'IT', 'name': 'Italy'}, {'iso_...",5.5,6
11338,46774,Chapayev,"Comedy,Drama,History,War",Released,This film is based on the book about Vasili Iv...,0.472296,ru,"[{'name': 'Lenfilm Studio', 'id': 27593}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",4.8,5


In [222]:
movies = movies[['id', 'title','genres', 'status', 'overview', 'popularity', 'original_language', 'vote_average', 'vote_count']]

In [234]:
movies.sample(5)

,id,title,genres,status,overview,popularity,original_language,vote_average,vote_count
23111,145220,Muppets Most Wanted,"Comedy,Adventure,Crime,Family",Released,"While on a grand world tour, The Muppets find ...",7.244956,en,6.2,318
11199,26502,Seven Men from Now,"Action,Adventure,Drama,Western",Released,Aging lawman Ben Stride burns a trail of murde...,2.511267,en,6.8,20
10351,16374,Sometimes in April,"Drama,History,War",Released,"Debra Winger, Oris Erhuero and Idris Elba star...",3.687491,en,7.2,16
7730,9890,The Stepford Wives,"Action,Comedy,Science Fiction",Released,"What does it take to become a Stepford wife, a...",10.815520,en,5.4,339
10506,23196,South Central,"Crime,Drama",Released,A man is put to prison for 10 years. Coming ou...,2.999450,en,6.6,20


In [241]:
movies.drop_duplicates(keep='first', inplace=True)

In [242]:
movies.to_csv('data\\movies_dataset.csv', index=False)

In [243]:
movies.isna().sum()


id                   0
title                0
genres               0
status               0
overview             0
popularity           0
original_language    0
vote_average         0
vote_count           0
dtype: int64

In [244]:
movies.duplicated().sum()

0

In [245]:
df = pd.read_csv('data\\ratings.csv')

In [246]:
df.duplicated().sum()

0

In [247]:
df.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [249]:
df.movieId.unique()

array([   110,    147,    858, ..., 173161, 174743, 175687], dtype=int64)